# NB This was eventually not used because it's lookups were even more messy

In [142]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep
from geopy import Point
import pycountry
from geopy.distance import geodesic
import numpy as np

In [2]:
looked_up_countries = {}

In [128]:
df = pd.read_csv("../docs/assets/mapdata.tsv", sep = "\t")
df

,Country,Project,Latitude,Longitude,SampleCount
0,Nigeria,54gene,10.600036,8.499972,449
1,South Africa,AWI-Gen,-26.216624,28.991639,100
2,Egypt,EGP,35.868130,-90.945675,11
3,Benin,H3Africa WGS,9.529347,2.258441,50
4,Botswana,H3Africa WGS,-23.168178,24.592874,48
...,...,...,...,...,...
66,Senegal,SGDP,14.475061,-14.452961,2
67,Sierra Leone,SGDP,8.640035,-11.840027,2
68,South Africa,SGDP,-28.816624,24.991639,2
69,Sudan,SGDP,10.900000,6.500000,2


In [129]:
def get_coordinates(row):
    if row.Country in looked_up_countries:
        print(f"'{row.Country}' (cached) = ".ljust(40), end = '')
        lat, long = looked_up_countries[row.Country]
        row["NewLatitude"] = lat
        row["NewLongitude"] = long
        print(f"{lat,long}")
        return row
    else:
        sleep(1)
        print(f"'{row.Country}' (looking up) = ".ljust(40), end = '')
        try:
            country_obj = pycountry.countries.get(name=row["Country"])
            geolocator = Nominatim(user_agent="h3abionet2.org")
            location = geolocator.geocode(country_obj.name, country_codes=[country_obj.alpha_2])
            row["NewLatitude"] = location.latitude
            row["NewLongitude"] = location.longitude
        except AttributeError:
            row["NewLatitude"] = None
            row["NewLongitude"] = None
        looked_up_countries[row.Country] = (row["NewLatitude"], row["NewLongitude"])
        print(f"{row['NewLatitude'], row['NewLongitude']}")
        return row

In [130]:
dfa = df.apply(get_coordinates, axis=1)

'Nigeria' (cached) =                    (9.6000359, 7.9999721)
'South Africa' (cached) =               (-28.8166236, 24.991639)
'Egypt' (cached) =                      (26.2540493, 29.2675469)
'Benin' (cached) =                      (9.5293472, 2.2584408)
'Botswana' (cached) =                   (-23.1681782, 24.5928742)
'Burkina Faso' (cached) =               (12.0753083, -1.6880314)
'Cameroon' (cached) =                   (4.6125522, 13.1535811)
'Cameroon' (cached) =                   (4.6125522, 13.1535811)
'Ghana' (cached) =                      (8.0300284, -1.0800271)
'Mali' (cached) =                       (16.3700359, -2.2900239)
'Nigeria' (cached) =                    (9.6000359, 7.9999721)
'Zambia' (cached) =                     (-14.5189121, 27.5589884)
'Afghanistan' (cached) =                (33.7680065, 66.2385139)
'Algeria' (cached) =                    (28.0000272, 2.9999825)
'Botswana' (cached) =                   (-23.1681782, 24.5928742)
'Brazil' (cached) =             

In [131]:
unknowns = set(dfa[dfa['NewLatitude'].isna()]["Country"])
print(unknowns)

for ctryname in unknowns:
    ctry = pycountry.countries.get(name=ctryname)
    geolocator = Nominatim(user_agent="h3abionet.org")
    loc = geolocator.geocode(ctryname)
    print(f"Fixing '{ctryname}'... =", loc.latitude, loc.longitude)
    dfa.loc[dfa["Country"]==ctryname,"NewLatitude"] = loc.latitude
    dfa.loc[dfa["Country"]==ctryname,"NewLongitude"] = loc.longitude

ctryname = "Palestine"
dfa.loc[dfa["Country"]==ctryname,"NewLatitude"] = 35.25 
dfa.loc[dfa["Country"]==ctryname,"NewLongitude"] = 32.0
    

{'Tanzania', 'Puerto Rico', 'Palestine', 'DRC', 'Russia', 'Vietnam'}
Fixing 'Tanzania'... = -6.5247123 35.7878438
Fixing 'Puerto Rico'... = 18.2247706 -66.4858295
Fixing 'Palestine'... = 31.7621153 -95.6307891
Fixing 'DRC'... = -2.9814344 23.8222636
Fixing 'Russia'... = 64.6863136 97.7453061
Fixing 'Vietnam'... = 15.9266657 107.9650855


In [132]:
duplicated = sorted(set(dfa[dfa["Country"].duplicated()]["Country"]))
sigma = 1
print(f"Jittering: ", end="")
for ctryname in duplicated:
    print(f"{ctryname}",end=", ")
    dfa.loc[dfa["Country"]==ctryname, "NewLatitude"] = dfa['NewLatitude'].apply(lambda x: x + np.random.normal(x, sigma, 1)[0])
    dfa.loc[dfa["Country"]==ctryname, "NewLongitude"] = dfa['NewLongitude'].apply(lambda x: x + np.random.normal(x, sigma, 1)[0])
print()

Jittering: Algeria, Botswana, Cameroon, Central African Republic, China, Colombia, DRC, Gambia, Italy, Japan, Kenya, Namibia, Nigeria, Pakistan, Sierra Leone, South Africa, United Kingdom, 


In [133]:
dfa[dfa["Country"].isin(duplicated)].sort_values(["Country"]).head()

,Country,Project,Latitude,Longitude,SampleCount,NewLatitude,NewLongitude
58,Algeria,SGDP,29.000027,2.999983,2,56.319366,6.160329
13,Algeria,HGDP,27.000027,1.999982,26,55.187509,7.468879
4,Botswana,H3Africa WGS,-23.168178,24.592874,48,-46.797569,47.889476
14,Botswana,HGDP,-23.168178,24.592874,4,-46.259699,47.181908
59,Botswana,SGDP,-23.168178,24.592874,2,-46.526884,48.995282


In [134]:
dfa.columns

Index(['Country', 'Project', 'Latitude', 'Longitude', 'SampleCount',
       'NewLatitude', 'NewLongitude'],
      dtype='object')

In [135]:
# df = pd.read_csv("../docs/assets/mapdata.tsv", sep = "\t")


# dfa["Latitude"] = dfa["NewLatitude"]
# dfa["Longitude"] = dfa["NewLongitude"]
# dfa = dfa[['Country', 'Project', 'Latitude', 'Longitude', 'SampleCount',]]

In [136]:
dfa.to_csv("../docs/assets/mapdata2.tsv", index=False, sep="\t")

In [137]:
dfa[dfa["Country"]=="Kenya"].sort_values(["Latitude","Longitude"])

,Country,Project,Latitude,Longitude,SampleCount,NewLatitude,NewLongitude
63,Kenya,SGDP,-1.241968,36.831398,9,2.227495,77.402076
45,Kenya,KGP,1.441968,38.431398,99,3.439947,76.849743


In [138]:
dfa[dfa["NewLongitude"].isna()]

,Country,Project,Latitude,Longitude,SampleCount,NewLatitude,NewLongitude


In [139]:
ctryname = "Algeria"
ctry = pycountry.countries.get(name=ctryname)
geolocator = Nominatim(user_agent="h3abionet.org")
loc = geolocator.geocode(ctryname, country_codes=[ctry.alpha_2])
print(f"Fixing '{ctryname}'... =", loc.latitude, loc.longitude)
# dfa.loc[dfa["Country"]==ctryname,"NewLatitude"] = loc.latitude
# dfa.loc[dfa["Country"]==ctryname,"NewLongitude"] = loc.longitude

Fixing 'Algeria'... = 28.0000272 2.9999825


In [140]:
ctry

Country(alpha_2='DZ', alpha_3='DZA', flag='🇩🇿', name='Algeria', numeric='012', official_name="People's Democratic Republic of Algeria")